In [57]:
import pandas as pd
import numpy as np
import operator
import string
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.data import load
from nltk.stem import SnowballStemmer
from string import punctuation
import re

#LEER EL DATASET

df=pd.read_csv('Dataset2.csv')
#L=np.asarray(df)
#print(L)

data = pd.read_csv("Dataset2.csv", sep=",")
##########################################
punctuation = list(string.punctuation)
stemmer = SnowballStemmer("spanish")
stop = stopwords.words('spanish') + punctuation
vc = TfidfVectorizer()
#############Documentos##################
Tweet = data[["Tweet"]]
T=np.asarray(Tweet)
bigNumOfWords = []
bigBagOfWords = []
unique = [' ']
bigTf = {}
bigTfidf= []
########################################
emoticons_str = r"""
    (?:
        [:=;] # :(
        [oO\-]? # No se
        [D\)\]\(\]/\\OpP] # 
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
##############Metodos####################



def generate_ngrams(s, n):
    # Convert to lowercases
    s = s.lower()
    # Romper la oración en el tokens
    tokens_first = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE )
    tokens = tokens_first.findall(s)
        
    # eliminar tokens vacíos, links, menciones, stopwords y hastags
    tokens = [token for token in tokens if (token != "" ) and (token not in stop) and (not token.startswith(('#', '@', 'https:', 'http:')))]
    #lematizar
    tokens = [stemmer.stem(token) for token in tokens]
    # funcion zip genera n-grams
    # Concatena los tokens en los ngrams y los retorna
    ngrams = zip(*[tokens[i:] for i in range(n)])
    return [" ".join(ngram) for ngram in ngrams]

def computeTF(wordDict, bagOfWords):
    tfDict = {}
    bagOfWordsCount = len(bagOfWords)
    for word, count in wordDict.items():
        if bagOfWordsCount != 0:
            tfDict[word] = count / float(bagOfWordsCount)
        else:
            tfDict[word] = 0

    return tfDict


def computeIDF(documents):
    import math
    N = len(documents)
    idfDict = dict.fromkeys(documents[0].keys(), 0)
    
    for document in documents:
        for word, val in document.items():
            
            if val > 0:
                idfDict[word] += 1
    for word, val in idfDict.items():
        if val != 0:
            idfDict[word] = math.log(N / float(val))
        else:
            idfDict[word] = 0
    return idfDict

def computeTFIDF(tfBagOfWords, idfs):
    tfidf = {}
    for word, val in tfBagOfWords.items():
        tfidf[word] = val * idfs[word]
    return tfidf

#########################################

##########Preprocesamiento##########
n=2
count = 0
count2 = 0
for tweet in T:
    bagOfWords = generate_ngrams(str(tweet), n)
    print(bagOfWords)
    unique = set(unique).union(set(bagOfWords))
    bigBagOfWords.append(bagOfWords)
############Contar Palabras######################
for tweet in bigBagOfWords:
    numOfWords = dict.fromkeys(unique, 0)
    for word in tweet:
        numOfWords[word] += 1
    bigNumOfWords.append(numOfWords)

count = 0
##################TF##########################
for tweet in T:
    tf = computeTF(bigNumOfWords[count], bigBagOfWords[count])
    bigTf[count] = tf
    count += 1
#####################IDF###################
idfs = computeIDF(bigNumOfWords)
###################TFIDF###################

for tweet in T:
    tfidf = computeTFIDF(bigTf[count2], idfs)
    bigTfidf.append(tfidf)
    count2 += 1
#################results############################33
df = pd.DataFrame(bigTfidf)
df.to_csv (r'TFIDF.csv', index = True, header=True)

['argument muestrem', 'muestrem dios', 'dios polit', 'polit habl', 'habl polici', 'polici muert', 'muert atent', 'atent santand', 'santand lleg', 'lleg vot', 'vot jep', 'jep favorec', 'favorec nin', 'nin victim', 'victim farc']
['constituyent arruin', 'arruin acuerd', 'acuerd paz', 'paz jep', 'jep asi', 'asi simpl']
['buen notici']
['seri sistem', 'sistem democrat', 'democrat separacion', 'separacion poder', 'poder president', 'president duqu', 'duqu gobiern', 'gobiern unic', 'unic respons', 'respons si', 'si vam', 'vam dialog', 'dialog hor', 'hor llam', 'llam cuent', 'cuent congres', 'congres pod', 'pod judicial', 'judicial jep', 'jep inclu', 'inclu ciudadan', 'ciudadan exig', 'exig trabaj', 'trabaj bien']
['si podri', 'podri dos', 'dos maner', 'maner aguil', 'aguil destru', 'destru jep', 'jep quem', 'quem hac', 'hac guerr', 'guerr ello', 'ello pediri', 'pediri urib', 'urib hag', 'hag constituyent', 'constituyent ataqu', 'ataqu venezuel', 'venezuel quem', 'quem derrit', 'derrit absorb

['usted mied', 'mied proces', 'proces paz', 'paz jep', 'jep sol', 'sol van', 'van destap', 'destap ollas', 'ollas podr', 'podr sin', 'sin usted', 'usted viv', 'viv guerr', 'guerr violenci', 'violenci sirv', 'sirv pais', 'pais llen', 'llen mied', 'mied negoci']
['dic usted', 'usted ahh', 'ahh buen', 'buen graci', 'graci dat', 'dat digal', 'digal mism', 'mism jep', 'jep part', 'part polit', 'polit farc', 'farc digal', 'digal mism', 'mism acuerd', 'acuerd paz', 'paz pas', 'pas tom', 'tom pod', 'pod mat', 'mat nadi', 'nadi contrari', 'contrari gobiern', 'gobiern uri']
['si pas', 'pas colombi', 'colombi segur', 'segur polici', 'polici estari', 'estari esper', 'esper turn', 'turn jep', 'jep ser', 'ser juzg']
['rat entregat', 'entregat jep']
['ardid llaver', 'llaver tranquil', 'tranquil espos', 'espos aun', 'aun contrat', 'contrat jep', 'jep bien', 'bien jugos', 'jugos coronel', 'coronel dig']
['adivin sin', 'sin desarroll', 'desarroll plan', 'plan cerc', 'cerc aniquil', 'aniquil part', 'part

In [45]:
print(T[1])
print(generate_ngrams(str(T[122]), 2))

['Con la constituyente arruinan el acuerdo de paz y la JEP, asi de simple']
['part malandr']


In [46]:
print(bigTfidf[1])

{' ': 0.0, 'bassil': 0.0, 'transicional': 0.0, 'das': 0.0, 'timochenk': 0.0, 'culp': 0.0, 'sol': 0.0, 'engan': 0.0, '500': 0.0, 'respuest': 0.0, 'extravi': 0.0, 'cint': 0.0, 'arrodill': 0.0, 'intermedi': 0.0, 'fumigacion': 0.0, 'artist': 0.0, 'pinch': 0.0, 'b': 0.0, 'banquer': 0.0, 'dialogonacional': 0.0, 'apuest': 0.0, 'person': 0.0, 'cultiv': 0.0, '6.748': 0.0, 'anuenci': 0.0, 'aprend': 0.0, 'deshonest': 0.0, 'jefesit': 0.0, '15': 0.0, 'holm': 0.0, 'aval': 0.0, 'cas': 0.0, 'parasit': 0.0, 'claroo': 0.0, 'polit': 0.0, 'imperfect': 0.0, 'pts': 0.0, 'alejandr': 0.0, 'vim': 0.0, 'despreci': 0.0, 'ajust': 0.0, 'adopt': 0.0, 'impresion': 0.0, 'fuerz': 0.0, 'ves': 0.0, 'beb': 0.0, 'absolut': 0.0, 'curs': 0.0, 'alto': 0.0, 'quep': 0.0, 'prest': 0.0, 'ataqu': 0.0, 'santamari': 0.0, 'petrism': 0.0, 'exist': 0.0, 'definit': 0.0, 'pit': 0.0, 'asom': 0.0, 'prostitu': 0.0, 'dec': 0.0, 'hac': 0.0, 'expedient': 0.0, 'ambiental': 0.0, 'rubr': 0.0, 'epa': 0.0, 'aument': 0.0, 'licenci': 0.0, 'empujon':

In [44]:
print(bigBagOfWords[1])
print(bigBagOfWords[416])
print(bigBagOfWords[428])

['constituyent', 'arruin', 'acuerd', 'paz', 'jep', 'asi', 'simpl']
['mes', 'dialog', 'deb', 'hab', 'represent', 'puebl', 'eligi', 'tambi', 'deb', 'explic', 'mas', 'farc', 'mas', 'jep', 'mas', 'band', 'congres', 'nin', 'primer', 'mas', 'tem', 'buen', 'dialog', 'inclu']
['pes', 'president', 'hac', 'entreg', 'march', 'mas', 'farc', 'pas', 'report', 'plebiscit', 'pas', 'rob', 'ahor', 'eligi', 'cumpl', 'dich', 'campan', 'mas', 'jep', 'fumigacion', 'extrad']


In [40]:
dic={}
con=0
for tweet in bigBagOfWords:
    for word, val in bigTfidf[con].items():
        if val > 0:
            dic.setdefault(word,[]).append(con)
    con += 1


In [56]:
print(len(dic['constituyent']))

41
